Including libraries and scripts

In [ ]:
%matplotlib inline
import nbimport
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import scipy.sparse.linalg as spla
np.set_printoptions(linewidth=130)

In [ ]:
from fsmfuncs import *
from ibmfuncs import *
from gridfuncs import *

Numerical grid for the fluid

In [ ]:
s1 = stretching(256, 0.033, 0.20, int(0.65/0.033), 16, 16, 0.04)[0]
s2 = stretching(128, 0.033, 0.20, int(0.65/0.033), 16, 16, 0.04)[0]
x = np.concatenate([-s2[::-1], s1[1:]])
s = stretching(192, 0.033, 0.20, int(0.65/0.033), 16, 16, 0.04)[0]
y = np.concatenate([-s[::-1], s[1:]])

n, m = len(y)-1, len(x)-1

dy, dx = np.diff(y), np.diff(x)
dxmin = min(np.min(dx), np.min(dy))

# Pressure
yp, xp = 0.5*(y[1:] + y[:-1]), 0.5*(x[1:] + x[:-1])
dyp, dxp = np.diff(yp), np.diff(xp)
p = np.zeros( (n, m) )

# Horizontal velocity
yu, xu = yp, x[1:-1]
u = np.zeros( (n, m-1) )

# Vertical velocity
yv, xv = y[1:-1], xp
v = np.zeros( (n-1, m) )

Immersed boundary

In [ ]:
r_ = 0.5
l = int((2*np.pi*r_)/dxmin)
ang_ = 2*np.pi*np.arange(l)/l

# Coordinates
xi = 0 + r_*np.cos(ang_)
eta = 0 + r_*np.sin(ang_)

ds = 2*np.pi*r_/l*np.ones(l)

# Velocity
uB = np.zeros_like(xi)
vB = np.zeros_like(xi)

Plot grid and immersed boundary

In [ ]:
plt.figure(figsize=(8,8))
X, Y = np.meshgrid(x, y)
plt.plot(X, Y, 'b-');
plt.plot(X.T, Y.T, 'b-');
plt.plot(xi, eta, 'ro-');
plt.axis('equal')

Boundary conditions + initial flow 

In [ ]:
uS, uN = np.ones(m-1), np.ones(m-1)
uE, uW = np.ones(n), np.ones(n)

vS, vN = np.zeros(m), np.zeros(m)
vE, vW = np.zeros(n-1), np.zeros(n-1)

u[:,:]=1
v[:,:]=0

Build matrices (I)

In [ ]:
G, DuW, DuE, DvS, DvN = gradient(dxp, dyp)
R, iR = weight (dx, dy)
Mh, iMh = mass_hat (dxp, dyp)
Lh, Lux0, Lux1, Luy0, Luy1, Lvx0, Lvx1, Lvy0, Lvy1 = laplacian_hat(dx, dy, dxp, dyp)
Eh = interpolation_hat(xi, eta, ds, x, y, xp, yp, dx, dy, dxp, dyp)
Hh = regularization_hat(xi, eta, ds, x, y, xp, yp, dx, dy, dxp, dyp)

E = Eh.dot(iR)
H = Mh.dot(Hh)

L = Mh.dot(Lh.dot(iR))

M = Mh.dot(iR)
iM = R.dot(iMh)

EET = E.dot(E.T)
EH = E.dot(H).tocsc()
iEH = spla.factorized(EH)

iML = iM.dot(L)
Q = sp.hstack([G, E.T])

Build matrices (II)

In [ ]:
#Reynolds number
iRe = 1/200.0

#Time step
dt = 0.30 * min(dxmin**2/iRe, dxmin)

print(dt, dxmin**2/iRe, dxmin)

In [ ]:
A = (M/dt - 0.5*iRe*L).tocsc()
B = (M/dt + 0.5*iRe*L).tocsr()

#Factorization step
iA = spla.factorized(A)

BN = dt*iM + (0.5*iRe)*dt**2*iML.dot(iM) + (0.5*iRe)**2*dt**3*iML.dot(iML.dot(iM))
QTBNQ = Q.T.dot(BN.dot(Q)).tocsc()
iQTBNQ = spla.factorized(QTBNQ)

In [ ]:
#Velocity flux
q = R.dot(np.concatenate([u.ravel(), v.ravel()]))
qast = q.copy()

Num1, Nvm1 = advection_hat(dx, dy, dxp, dyp, iR.dot(q),  uS, uN, uW, uE, vS, vN, vW, vE)
Nu, Nv = Num1, Nvm1

Time loop 

In [ ]:
nt = int(100/dt)
print("Performing", nt, "steps")

residuals = np.zeros(nt)

# Drag and lift coefficients
CFx = np.zeros(nt)
CFy = np.zeros(nt)

for k in range(nt):    
    ru = iRe*(Lux0.dot(uW) + Lux1.dot(uE) + Luy0.dot(uS) + Luy1.dot(uN)) - 1.5*Nu + 0.5*Num1
    rv = iRe*(Lvx0.dot(vW) + Lvx1.dot(vE) + Lvy0.dot(vS) + Lvy1.dot(vN)) - 1.5*Nv + 0.5*Nvm1
    
    bc1 = Mh.dot(np.concatenate([ru, rv]))
    r1 = B.dot(q.ravel()) + bc1
    
    # Resolution of the first equation
    qast = iA(r1)
    
    bc2 = - (DuW.dot(uW*dy) + DuE.dot(uE*dy) + DvS.dot(vS*dx) + DvN.dot(vN*dx))
    r2 = np.concatenate([-bc2, uB, vB])
    
    # Resolution of the second equation
    λ = iQTBNQ(Q.T.dot(qast) - r2)

    # Projection step
    qp1 = qast - BN.dot(Q.dot(λ))
    
    # Residuals
    residuals[k] = la.norm(qp1-q)/(dt*la.norm(qp1))
    
    # Forcing term 
    F = iMh.dot(E.T.dot(λ[n*m:]))
    Fx, Fy = F[:n*(m-1)].reshape((n, m-1)), F[n*(m-1):].reshape((n-1,m))
    CFx[k] = 2*np.sum(Fx*dxp[np.newaxis,:]*dy[:,np.newaxis])
    CFy[k] = 2*np.sum(Fy*dx[np.newaxis,:]*dyp[:,np.newaxis])    
    
    if k%100==0:
        print(k, k*dt, residuals[k], CFx[k], CFy[k])
    
    q = qp1
    uE = uE - dt/dx[-1]*(uE - iR.dot(q)[:n*(m-1)].reshape((n, m-1))[:,-1])
    
    Num1, Nvm1 = Nu, Nv
    Nu, Nv = advection_hat(dx, dy, dxp, dyp, iR.dot(q), uS, uN, uW, uE, vS, vN, vW, vE)


Velocity, pressure, vorticity and forcing term

In [ ]:
iRq = iR.dot(q)
u, v = iRq[:n*(m-1)].reshape((n, m-1)), iRq[n*(m-1):].reshape((n-1, m))
p = λ[:n*m].reshape((n,m))
f = λ[n*m:]
w = np.diff(v,axis=1)/dxp[np.newaxis,:]-np.diff(u,axis=0)/dyp[:,np.newaxis]

Display solution

In [ ]:
# Ploting velocity
x0, x1 = -2, 7
y0, y1 = -4.5, 4.5
plt.figure(figsize=(5.5*3,4))
plt.subplot(1,3,1)
plt.pcolormesh(xu, yu, u, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

plt.subplot(1,3,2)
plt.pcolormesh(xv, yv, v, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

# Ploting pressure
plt.subplot(1,3,3)
plt.pcolormesh(xp, yp, p, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

In [ ]:
# Ploting vorticity
plt.figure(figsize=(5.5,4))
plt.pcolormesh(xu, yv, w, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

In [ ]:
#Ploting drag coefficient
plt.plot(np.arange(len(CFy))*dt, CFx)
plt.xlim(0, 100)
plt.ylim(0.8, 1.40)
plt.xlabel('t')
plt.ylabel('C_l')